# Part 2 - Fine Tune LLM
- Fine tune LLM on our synthetically generated dataset to generate domain names from descriptions.
- Do preliminary evaluation using various standard NLP metrics such as rouge, bleu, levenshtein, etc.
- Due to time and Kaggle storage constraints, we'll just run this script changing different hyperparameters by hand as opposed to running a for loop over multiple configs, which would lead to a huge amount of space being used that will exceed the output allowance.
- We will ultiamtely vary:
    - Batch Size
    - Learning Rate
    - Switch on/off LoRA and PEFT

In [1]:
VERSION = 'v1'
FLAG = 'small'

In [2]:
!pip install -q transformers torch peft rouge_score levenshtein

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.4 MB/s eta 0:00:00


In [3]:
import time
import pandas as pd
import json

In [4]:
# Load in data
df = pd.read_csv('/kaggle/input/domain-name-generator/data/synthetic_data-small.csv')
df.head()

,business_description,domain_name,category
0,A mobile app that helps people find local farm...,freshfinds.app,NaN
1,An online subscription service for eco-friendl...,greenbundle.com,NaN
2,A platform that helps small businesses adverti...,smallbusiness.co,NaN
3,A website that connects pet owners with local ...,petsitter.com,NaN
4,A blog platform for creative writers to share ...,writeon.com,NaN


In [5]:
# Create dataset object for training
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType
import torch
from sklearn.model_selection import train_test_split

2025-07-30 14:37:20.138660: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753886240.314806      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753886240.367378      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# Combine into prompt: "Business: {description} -> Domain: {domain_name}" for autoregressie model
df["text"] = df.apply(lambda row: f"Business: {row['business_description']} -> Domain: {row['domain_name']}", axis=1)

# Split data set and turn into data set objects
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
train_df.to_csv('train.csv',index=False)
test_df.to_csv('test.csv',index=False)

# Add a stop tag to the text column
#train_df['text'] = train_df['text'] + '<|eot_id|>'
#test_df['text'] = test_df['text'] + '<|eot_id|>'

train_dataset = Dataset.from_pandas(train_df[["text"]])
test_dataset = Dataset.from_pandas(test_df[["text"]])

# Create dict of both training and testing for easy access in Trainer
datasets = DatasetDict({"train": train_dataset, "test": test_dataset})

In [7]:
# Explore a piece of dataset as sanity check
print(datasets['train'][0]['text'])

Business: A subscription box service that delivers healthy snacks and wellness products to your home. -> Domain: wellnessbox.com


In [8]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

In [9]:
model_name_or_path = "/kaggle/input/llama-3.2/transformers/1b-instruct/1"
#tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct", 
    trust_remote_code=True,
    use_auth_token=hf_token)

#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add a safe pad token
tokenizer.pad_token = "<|eot_id|>"  # if not already set
tokenizer.eos_token = "<|eot_id|>"

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [10]:
# Load base model and tokenize
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    #torch_dtype=torch.bfloat16,
    #torch_dtype=torch.float16,
    device_map="auto" # put onto cuda automatically if available
)
model.resize_token_embeddings(len(tokenizer))         # Resize embedding to include [PAD]
model.config.eos_token_id = tokenizer.eos_token_id

def tokenize(example):
    result = tokenizer(
        example["text"], 
        truncation=True, 
        padding=True,
        #padding="max_length", 
        max_length=128
    )
    result['labels'] = result['input_ids'].copy()
    return result

tokenized_dataset = datasets.map(tokenize, batched=True)

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [11]:
# Check one input after tokenizing
print(tokenizer.special_tokens_map)

{'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>', 'pad_token': '<|eot_id|>'}


In [12]:
# Create LoRA (parameter-efficient fine-tuning) and wrap model in it
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Commenting out removes peft and does full fine tuning
model = get_peft_model(model, peft_config)

In [13]:
# Create data collator to patch, dynamically pad, etc.
data_collator = DataCollatorForLanguageModeling(
    tokenizer, 
    mlm=False
)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [14]:
# Create training arguments, data-collator ()

training_args = TrainingArguments(
    output_dir="./outputs",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    #fp16=True,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
    report_to="none",
    max_grad_norm=1.0,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

/tmp/ipykernel_19/3942653682.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [15]:
# Look at a few inputs pre training
for i in range(1):
    print(tokenizer.decode(tokenized_dataset["train"][i]["input_ids"]))

<|begin_of_text|>Business: A subscription box service that delivers healthy snacks and wellness products to your home. -> Domain: wellnessbox.com<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|>


In [16]:
trainer.train()

# Save final model
trainer.save_model(f"./fine-tuned-llama-domain-generator-{VERSION}")
tokenizer.save_pretrained(f"./fine-tuned-llama-domain-generator-{VERSION}")

Step,Training Loss
10,3.846000
20,3.488400
30,3.235100


('./fine-tuned-llama-domain-generator-v1/tokenizer_config.json',
 './fine-tuned-llama-domain-generator-v1/special_tokens_map.json',
 './fine-tuned-llama-domain-generator-v1/chat_template.jinja',
 './fine-tuned-llama-domain-generator-v1/tokenizer.json')

#### Checks Before Eval

In [17]:
# Generate a sample prompt from the test set
sample_prompt = datasets['test'][0]['text'].split("-> Domain:")[0].replace("Business:", "").strip()
sample_true_domain = datasets['test'][0]['text'].split("-> Domain:")[-1].strip()
sample_prompt

'A subscription box service for unique, handmade jewelry.'

In [18]:
datasets['test'][0]['text']

'Business: A subscription box service for unique, handmade jewelry. -> Domain: handmadebox.com'

In [19]:
# Put into tokenizer
inputs = tokenizer(sample_prompt, return_tensors="pt").to(model.device)
print("Max token ID:", inputs["input_ids"].max().item())
print("Model vocab size:", model.config.vocab_size)
print(inputs)

Max token ID: 128000
Model vocab size: 128256
{'input_ids': tensor([[128000,     32,  15493,   3830,   2532,    369,   5016,     11,  52786,
          31817,     13]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [20]:
# Check to make sure we're not having token id mis matches
print("pad_token_id:", tokenizer.pad_token_id)
print("eos_token_id:", tokenizer.eos_token_id)
print("vocab size:", model.config.vocab_size)

pad_token_id: 128009
eos_token_id: 128009
vocab size: 128256


In [21]:
'''
model_id = "/kaggle/input/llama-3.2/transformers/1b-instruct/1"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct", 
    trust_remote_code=True,
    use_auth_token=hf_token)

prompt = "Business: dog walking service -> Domain:"
inputs = tokenizer(sample_prompt, return_tensors="pt")
inputs = {k: v.to(model.device) for k, v in inputs.items()}

model.eval()
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=10,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
print(tokenizer.decode(outputs[0]))
'''

'\nmodel_id = "/kaggle/input/llama-3.2/transformers/1b-instruct/1"\nmodel = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)\ntokenizer = AutoTokenizer.from_pretrained(\n    "meta-llama/Llama-3.2-1B-Instruct", \n    trust_remote_code=True,\n    use_auth_token=hf_token)\n\nprompt = "Business: dog walking service -> Domain:"\ninputs = tokenizer(sample_prompt, return_tensors="pt")\ninputs = {k: v.to(model.device) for k, v in inputs.items()}\n\nmodel.eval()\nwith torch.no_grad():\n    outputs = model.generate(\n        **inputs,\n        max_new_tokens=10,\n        do_sample=False,\n        pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,\n        eos_token_id=tokenizer.eos_token_id,\n    )\nprint(tokenizer.decode(outputs[0]))\n'

## Evaluating model
- We do have some eval metrics above that computes "language model loss" which measures model's ability to predict the next token, not necessarily how relevant/quality the output is.
- Thus, we implement some scoring
    - ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a set of metrics used to evaluate the quality of machine-generated summaries by comparing them to human-written reference summaries.
    - In general, a ROUGE score above 0.5 is often considered good, especially for ROUGE-1, while scores below 0.2 are often considered poor.
    - A good BLEU (bilingual evaluation understudy) score is above 0.3...however, this isn't the best use case for BLEU since its primarily used for tranlsation.
    - A good levenshtein distance is not a fixed thing, but 0 represents identical strings.

In [22]:
from rouge_score import rouge_scorer
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import Levenshtein
import re

In [23]:
def extract_domain(text: str) -> str:
    """
    Extracts the first domain-like string from the LLM output.
    Handles formats like:
    - plain: domain.com
    - markdown: [domain.com](http://domain.com)
    - prefixed: 'Website: domain.com'
    - continuation: '-> Tagline: ...'
    """

    # 1. Try to extract markdown-style domain
    markdown_match = re.search(r'\[([a-zA-Z0-9\.-]+\.[a-z]{2,})\]\(http[^\)]*\)', text)
    if markdown_match:
        return markdown_match.group(1)

    # 2. Find domain-like text directly
    domain_match = re.search(r'\b([a-zA-Z0-9-]+\.[a-z]{2,})\b', text)
    if domain_match:
        return domain_match.group(1)

    return "NO_DOMAIN_FOUND"

In [24]:
# Create function to check if domain name is valid domain name
VALID_TLDS = ['.com', '.ai', '.io', '.net', '.org', '.co', '.app', '.tech', '.dev']

def is_valid_domain(domain: str) -> bool:
    domain = domain.strip().lower()

    # Basic structure check: must contain a valid TLD
    if not any(domain.endswith(tld) for tld in VALID_TLDS):
        return False

    # No whitespace
    if ' ' in domain:
        return False

    # Length check
    if len(domain) > 63 or len(domain) == 0:
        return False

    # Valid characters (alphanumeric, hyphen, and dot)
    pattern = r'^[a-z0-9\-\.]+$'
    if not re.match(pattern, domain):
        return False

    # Should not contain multiple dots (e.g., no sentences)
    if domain.count('.') > 1:
        return False

    return True

In [25]:
# get rouge scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
scores = []
bleu_scores = []
levenshtein_distances = []
brandability_scores = []
predictions = []
valid_domains = []

In [26]:
# Put pytorch model in eval mode
model.eval()

# Loop over test data
for item in test_dataset:
    input_text = item["text"]
    if "-> Domain:" not in input_text:
        continue
    description = input_text.split("-> Domain:")[0].replace("Business:", "").strip()
    true_domain = input_text.split("-> Domain:")[-1].strip()

    prompt = f"Business: {description} -> Domain:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=20, 
            do_sample=True, 
            top_k=50, 
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id)
    
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
    #pred_domain = output_text.split("-> Domain:")[-1].split("<|")[0].split("\n")[0].strip()
    #pred_domain = extract_domain(pred_domain)
    pred_domain = decoded.split("-> Domain:")[-1].strip()

    # Safety filter
    banned_keywords = ["sex", "kill", "drugs", "hate", "murder",
                        "adult-content", 'adult content', "explicit", 
                        "porn", "xxx", "nude", "erotic", "pornography"
                        "gambling", "casino", "hate", "violence"]
    if any(bad in pred_domain.lower() for bad in banned_keywords):
        pred_domain = "[REDACTED: Unsafe Output]"

    # Check if valid domain       
    valid_domain = is_valid_domain(pred_domain)
    valid_domains.append(valid_domain)
    
    # ROUGE
    rouge_score = scorer.score(true_domain, pred_domain)["rougeL"].fmeasure
    scores.append(rouge_score)

    # BLEU
    bleu = sentence_bleu([true_domain.split(".")], pred_domain.split("."), smoothing_function=SmoothingFunction().method1)
    bleu_scores.append(bleu)

    # Levenshtein
    lev_distance = Levenshtein.distance(true_domain, pred_domain)
    levenshtein_distances.append(lev_distance)

    # Brandability heuristic: short, no numbers/symbols, vowel-consonant balance
    def brandability(domain):
        score = 1.0
        if len(domain) > 15: score -= 0.3
        if any(char.isdigit() or not char.isalnum() for char in domain): score -= 0.3
        vowels = sum(1 for c in domain if c in "aeiou")
        consonants = sum(1 for c in domain if c.isalpha() and c not in "aeiou")
        ratio = vowels / (consonants + 1)
        if ratio < 0.2 or ratio > 0.8: score -= 0.2
        return max(0, round(score, 2))

    brandability_score = brandability(pred_domain)
    brandability_scores.append(brandability_score)

    predictions.append({
        "description": description,
        "true": true_domain,
        "pred": pred_domain,
        "rougeL": rouge_score,
        "bleu": bleu,
        "levenshtein": lev_distance,
        "brandability": brandability_score,
        "is_valid_domain": valid_domain,
    })

# Save predictions
results_df = pd.DataFrame(predictions)
results_df.to_csv(f"predictions_eval-{VERSION}-{FLAG}.csv", index=False)

# Report average scores
print_string = f"Pct of Predictions that are valid domains = {results_df['is_valid_domain'].mean():.2%}"
print(print_string)

Pct of Predictions that are valid domains = 0.00%
